In [ ]:
import numpy as np
import pandas as pd

from shapely import Polygon, LineString, Point
from shapely.ops import unary_union
from shapely import wkt

import utm
import ast

In [ ]:
def from_latlon(latitude, longitude, geozone_num, geozone_let):
    """
    convert lat, lon to (x, y) coordinates (in meters)
    :param latitude:  list or np.array (shape: (n,))
    :param longitude: list or np.array (shape: (n,))
    :param geozone_num: geozone integer
    :param geozone_let: geozone letter
    :return: np.array of (x, y) coordinates (shape: (n, 2))
    """
    xy = np.array([
        utm.from_latlon(
            latitude=lat, 
            longitude=lon, 
            force_zone_number=geozone_num,
            force_zone_letter=geozone_let
        )[:2] for lat, lon in zip(latitude, longitude)
    ])
    return xy



def get_subfields_xy(field_bounds: list, geozone_num, geozone_let):
    """
    convert GPS bounds of field into (x, y) coordinates (in meters)
    :param field_bounds: list with GPS bounds of subfields
    :return: list of (x, y) bounds of the subfields
    """
    subfields = [np.array(subfield).T for subfield in field_bounds]
    bounds_xy = [from_latlon(field[1], field[0], geozone_num, geozone_let) for field in subfields]
    return bounds_xy



def get_bounds_from_string(field_bounds_str : str):
    """
    convert string representation of the bounds into list
    :param field_bounds_str: string representation of the bounds
    :return: list of bounds of the subfields
    """
    if field_bounds_str[:4] == '[[[[':
        field_bounds_str = field_bounds_str.replace('[[[', '[[').replace(']]]', ']]')
    field_bounds = ast.literal_eval(field_bounds_str) 
    return field_bounds



def get_geozone(latitude, longitude):
    """
    :param latitude: 
    :param longitude: 
    :return: UTM geozone number and letter
    """
    geozone_num, geozone_let = utm.from_latlon(latitude, longitude)[2:]
    return geozone_num, geozone_let



def get_polygon_from_str_bounds(field_bounds_str, geozone_num, geozone_let):
    """
    get field polygon from string representation of its bounds
    :param field_bounds_str: string representation of the bounds of the field
    :param geozone_num: utm geozone number
    :param geozone_let: utm geozone letter
    :return: shapely field polygon
    """
    # get GPS field bounds from string representation
    field_bounds = get_bounds_from_string(field_bounds_str)
    # convert GPS field bounds to XY and create field_polygon
    subfields_xy = get_subfields_xy(field_bounds, geozone_num, geozone_let)
    field_polygon = unary_union([
        Polygon(subfield).buffer(0) if len(subfield) > 1 else None for subfield in subfields_xy
    ])
    return field_polygon



def denoise_track(track_xy, simplification_m):
    """
    get denoised track 
    :param track_xy: XY-coordinates of the original track
    :param simplification_m: simplification tolerance
    :return: denoised track 
    """
    # create track LineString object
    track = LineString(track_xy)
    # simplify the track
    simplified_track = track.simplify(simplification_m)
    # project each point of the original track onto the simplified track
    projected_points = [
        simplified_track.interpolate(
            simplified_track.project(
                Point(x, y)
            )
        )
        for x, y in track_xy
    ]
    return projected_points

# Main function : area and distance calculator

In [ ]:
def get_area_and_distance(
    tool_width : float,
    gps_m_deviation : float,
    
    time : list,
    track_lat : list,
    track_lon : list,
    
    field_bounds : list,
    
    encoded_field_polygon : str = None,
    encoded_calculated_track : str = None,
    
    gps_geozone_num : int = None,
    gps_geozone_let : str = None,
    
    track_polygon_simplification_m = 0.3
):
    """
    Calculates path distances and processed field area

    Parameters
    ----------
    tool_width : float
        Tool-width in meters.
    gps_m_deviation : float 
        GPS deviation in meters.
    time : array-like
        List of timestamps.
    track_lat : array-like
        List of track latitudes.
    track_lon : array-like
        List of track longitudes.
    field_bounds : list of lists
        List of lists with raw GPS field bounds !(lon, lat)!
    encoded_field_polygon : str
        Encoded version of the polygon in XY-system.
    encoded_calculated_track : str
        Encoded version of the calculated track in XY-system.
    gps_geozone_num : int
        GPS geozone number (UTM).
    gps_geozone_let : str
        GPS geozone letter (UTM).
    simplification_m : float
        Tolerance for the simplification of the calculated track.
    
    Returns
    -------
    distance_and_area_calculator : dict
        Dict with list of path distances, areas, and intersection areas,
        and updated parameters of the track and field:
            encoded_field_polygon,
            encoded_calculated_track,
            gps_geozone_num,
            gps_geozone_let.

    """
    
    # init field_polygon
    field_polygon = None
    
    # calculate track buffer as the sum of tool-width and GPS deviation
    buffer = (tool_width + gps_m_deviation) / 2
    
    # if GPS geozone is not defined
    if gps_geozone_num is None or gps_geozone_let is None:
        # define geozone num and let
        gps_geozone_num, gps_geozone_let = get_geozone(track_lat[0], track_lon[0])
        # get field polygon from string bounds
        field_polygon = get_polygon_from_bounds(field_bounds, gps_geozone_num, gps_geozone_let)
        # simplify the field polygon to fasten the computations
        field_polygon = field_polygon.simplify(track_polygon_simplification_m*2)
        # create encoded version of the field polygon
        encoded_field_polygon = field_polygon.wkt
        
    # convert GPS coordinates of the track 
    track_xy = from_latlon(track_lat, track_lon, gps_geozone_num, gps_geozone_let)
    
    # if the field polygon isn't defined
    if field_polygon is None:
        # and its encoded version isn't defined
        if encoded_field_polygon is None:
            # get field polygon from string bounds
            field_polygon = get_polygon_from_bounds(field_bounds, gps_geozone_num, gps_geozone_let)
            # simplify the field polygon to fasten the computations
            field_polygon = field_polygon.simplify(track_polygon_simplification_m*2)
            # create encoded version of the field polygon
            encoded_field_polygon = field_polygon.wkt
        else:
            # load field polygon from encoded version
            field_polygon = wkt.loads(encoded_field_polygon)
    
    
    # dublicate the first point of the track
    track_xy = np.concatenate([track_xy[:1], track_xy])
    
    # denoise track 
    track_xy = denoise_track(track_xy, gps_m_deviation)
    

    # if the encoded calculated track representation isn't defined
    if encoded_calculated_track is None:
        # create empty calculated track 
        calculated_track = LineString([])
    else:
        # load calculated track from encoded version
        calculated_track = wkt.loads(encoded_calculated_track)
        
        
    # create path distance array, cumulative area array, and intersection area array
    path_distance = np.zeros(len(time))
    cumulative_field_processed = np.concatenate([[calculated_track.area], np.zeros(len(time))])
    intersection_area = np.zeros(len(time))
    
    # create empty last processed subtrack to calculate intersection areas
    last_subtrack = Polygon(None)
    

    # calculation loop : for two consecutive track points (i, i+1)
    # !! ADD index_from !!
    for i in range(len(time)):
        # create LineString object as a subtrack
        subtrack = LineString(track_xy[i : i+2])
        # compute distance between these points, and add to the list
        path_distance[i] = subtrack.intersection(field_polygon).length

        # if the vehicle has no tool, continue calculating only path distance
        if tool_width == 0:
            continue
        
        # if the vehicle has the tool, calculate processed area
        
        # add buffer to the subtrack, and intersect it with the field polygon
        subfield = subtrack.buffer(buffer) \
                           .intersection(field_polygon) 
        
        # calculate intersection area
        intersection_area[i] = subfield.difference(last_subtrack) \
                                       .intersection(calculated_track) \
                                       .area
        
        
        # add this intersection to the preprocessed track
        calculated_track = calculated_track.union(subfield)
        
        # create simplified track to fasten the computations
        simplified = calculated_track.simplify(track_polygon_simplification_m, preserve_topology=False)
        # if the difference in areas is small, use simplified track as a new calculated track
        if np.abs(simplified.area - calculated_track.area) < (tool_width * gps_m_deviation) and cumulative_field_processed[i] < simplified.area:
            calculated_track = simplified
            
        # add new cumulative area to the list
        cumulative_field_processed[i+1] = calculated_track.area   

    # calculate processed area for each timestamp
    field_processed = cumulative_field_processed[1:] - cumulative_field_processed[:-1]

    # create output dictionary
    distance_and_area_calculator = {
        'encoded_field_polygon' : encoded_field_polygon,
        'encoded_calculated_track' : calculated_track.wkt,

        'gps_geozone_num' : gps_geozone_num,
        'gps_geozone_let' : gps_geozone_let,
        
        'path_distance' : path_distance,
        'field_processed' : field_processed,
        'intersection_area' : intersection_area
    }
    
    return distance_and_area_calculator

# Test

In [ ]:
df = pd.read_csv('../fields-data/new_fields_500.csv')
field_bounds_str = df.iloc[5]['geometry_coordinates']
field_bounds_str[:30]

In [ ]:
bound_path = np.array(get_bounds_from_string(field_bounds_str)[0])[:,:2]
bound_path2 = np.array(get_bounds_from_string(field_bounds_str)[1])[:,:2]
LineString(bound_path).union(LineString(bound_path2))

In [ ]:
len(bound_path)

In [ ]:
# start_point = np.array([50.067829, 23.983841])[::-1]
path = np.concatenate([
    np.linspace([50.041046, 24.544801], [50.040669, 24.548822], 300),
    np.linspace([50.040497, 24.548638], [50.040834, 24.544704], 300),
    np.linspace([50.040605, 24.544640], [50.040235, 24.548363], 300),
    np.linspace([50.040019, 24.548111], [50.040353, 24.544593], 300),
    np.linspace([50.040113, 24.544453], [50.039806, 24.547839], 300),
    np.linspace([50.039593, 24.547494], [50.039902, 24.544301], 300),
    np.linspace([50.039763, 24.544219], [50.039473, 24.547330], 300)
])


diff_lat = np.random.uniform(-0.000002, 0.000002, 2100)*2
diff_lon = np.random.uniform(-0.000001, 0.000001, 2100)*2
# diff_lon = np.zeros(2100)

# path = [start_point]
# for dx, dy in zip(diff_x, diff_y):
#     path.append(path[-1] + np.array([dx, dy]))
# path = np.array(path)

# linepath = LineString(path)
# linepath.union(LineString(bound_path))
path = path + np.array([diff_lat, diff_lon]).T

# Time test for 3 consecutive 300-coordinate tracks 

In [ ]:
path = path[:,::-1]

In [ ]:
# parameters
# path = bound_path

track_len = 300

tool_width = 0
time = np.ones(track_len)

track_lon, track_lat = list(path[:track_len].T)

In [ ]:
len(path)

In [ ]:
LineString(path+np.array([diff_lat, diff_lon]).T)

## First 300 coords

In [ ]:
%%time
first_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str
)

## Second 300 coords

In [ ]:
track_lon, track_lat = path[track_len : track_len*2].T
# track_lon, track_lat = path[track_len : ].T

In [ ]:
%%time
second_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=track_lat,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=first_300['encoded_field_polygon'],
    encoded_calculated_track=first_300['encoded_calculated_track'],
    gps_geozone_num=first_300['gps_geozone_num'],
    gps_geozone_let=first_300['gps_geozone_let'],
    last_point_xy=first_300['last_point_xy'],
    last_path_distance=first_300['path_distance'][-1],
)

## Third 300 coords

In [ ]:
track_lon, track_lat = path[track_len*2 :track_len*3].T
time = track_lon

In [ ]:
%%time
third_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=second_300['encoded_field_polygon'],
    encoded_calculated_track=second_300['encoded_calculated_track'],
    gps_geozone_num=second_300['gps_geozone_num'],
    gps_geozone_let=second_300['gps_geozone_let'],
    last_point_xy=second_300['last_point_xy'],
    last_path_distance=second_300['path_distance'][-1],
)

## Fourth 300 coords

In [ ]:
track_lon, track_lat = path[track_len*3 :track_len*4].T
time = track_lon

In [ ]:
%%time
fourth_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=third_300['encoded_field_polygon'],
    encoded_calculated_track=third_300['encoded_calculated_track'],
    gps_geozone_num=third_300['gps_geozone_num'],
    gps_geozone_let=third_300['gps_geozone_let'],
    last_point_xy=third_300['last_point_xy'],
    last_path_distance=third_300['path_distance'][-1],
)

## Fifth 300 coords

In [ ]:
track_lon, track_lat = path[track_len*4 :track_len*5].T
time = track_lon

In [ ]:
%%time
fifth_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=fourth_300['encoded_field_polygon'],
    encoded_calculated_track=fourth_300['encoded_calculated_track'],
    gps_geozone_num=fourth_300['gps_geozone_num'],
    gps_geozone_let=fourth_300['gps_geozone_let'],
    last_point_xy=fourth_300['last_point_xy'],
    last_path_distance=fourth_300['path_distance'][-1],
)

## Sixth 300 coords

In [ ]:
track_lon, track_lat = path[track_len*5 :track_len*6].T
time = track_lon

In [ ]:
%%time
sixth_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=fifth_300['encoded_field_polygon'],
    encoded_calculated_track=fifth_300['encoded_calculated_track'],
    gps_geozone_num=fifth_300['gps_geozone_num'],
    gps_geozone_let=fifth_300['gps_geozone_let'],
    last_point_xy=fifth_300['last_point_xy'],
    last_path_distance=fifth_300['path_distance'][-1],
)

## Seventh 300 coords

In [ ]:
track_lon, track_lat = path[track_len*6 :].T
time = track_lon

In [ ]:
%%time
seventh_300 = get_area_and_distance(
    tool_width=tool_width,
    gps_m_deviation=1,
    
    time=time,
    track_lat=track_lat,
    track_lon=track_lon,
    
    field_bounds_str=field_bounds_str,
    encoded_field_polygon=sixth_300['encoded_field_polygon'],
    encoded_calculated_track=sixth_300['encoded_calculated_track'],
    gps_geozone_num=sixth_300['gps_geozone_num'],
    gps_geozone_let=sixth_300['gps_geozone_let'],
    last_point_xy=sixth_300['last_point_xy'],
    last_path_distance=sixth_300['path_distance'][-1],
)

### Calculated track area comparison

In [ ]:
%%time
first_track = wkt.loads(first_300['encoded_calculated_track'])

In [ ]:
%%time
second_track = wkt.loads(second_300['encoded_calculated_track'])

In [ ]:
%%time
third_track = wkt.loads(third_300['encoded_calculated_track'])

In [ ]:
%%time
fourth_track = wkt.loads(fourth_300['encoded_calculated_track'])

In [ ]:
%%time
fifth_track = wkt.loads(fifth_300['encoded_calculated_track'])

In [ ]:
%%time
sixth_track = wkt.loads(sixth_300['encoded_calculated_track'])

In [ ]:
%%time
seventh_track = wkt.loads(seventh_300['encoded_calculated_track'])

In [ ]:
%%time
field_poly = wkt.loads(first_300['encoded_field_polygon'])

In [ ]:
first_track

In [ ]:
second_track

In [ ]:
third_track

In [ ]:
field_poly

In [ ]:
first_track.area, field_poly.intersection(first_track).area, first_300['path_distance'][-1]

In [ ]:
second_track.area, field_poly.intersection(second_track).area

In [ ]:
third_track.area, field_poly.intersection(third_track).area

In [ ]:
fourth_track.area, field_poly.intersection(fourth_track).area

In [ ]:
fifth_track.area, field_poly.intersection(fifth_track).area

In [ ]:
sixth_track.area, field_poly.intersection(sixth_track).area

In [ ]:
seventh_track.area, field_poly.intersection(seventh_track).area

In [ ]:
seventh_track

In [ ]:
fourth_300['path_distance']